##Read NETCDF Data

In [2]:
#Imports
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR 
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import sklearn.metrics as metric
from netCDF4 import Dataset
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt
import pandas as pd

#Some settings for plots
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
plt.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
plt.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['font.size'] = 18

In [3]:
#Load data
path = 'Data/ASO_Tuolumne.nc'
data=Dataset(path,mode='r')


D = np.transpose(data.variables['D'])
not_nan = np.full((D[:,:,0].shape), True, dtype=bool)#
"""Find values that are covered by all passes"""
for i in range(D.shape[2]):
    nans =~ np.isnan(D[:,:,i]) # A 2D Boolean array whose elements are true for the pixels that are not NaN.
    not_nan = not_nan*nans

def read_and_clean(variable,not_nan):
    """Read variables from netcdf data and create a 1D array for regression"""
    variable = data.variables[variable]
    variable = np.transpose(np.array(variable))  
    image = np.zeros_like(variable) # Image template array to map back to later.
    variable_image = image
    clean_image = variable[not_nan] # A 1D array that contains the non NaN points in z. 
    variable_image[not_nan] = clean_image # Set to z_clean for non NaN points.
    variable_image[~not_nan] = np.nan # Set to NaN for remaining points.
    return variable_image,clean_image

D,D_clean  = read_and_clean('D',not_nan)#SWE
z,z_clean  = read_and_clean('z',not_nan)
asp,asp_clean = read_and_clean('asp',not_nan)#aspect
northness = -np.cos(asp); northness_clean = -np.cos(asp_clean)
eastness = np.sin(asp);eastness_clean = np.sin(asp_clean)

DOY = np.array(data.variables['DOY'])
year = np.array(data.variables['year'])


slp,slp_clean = read_and_clean('slp',not_nan)
cc,cc_clean = read_and_clean('cc',not_nan)
X,X_clean  = read_and_clean('X',not_nan)
Y,X_clean  = read_and_clean('Y',not_nan)
svf,svf_clean = read_and_clean('svf',not_nan)#sky view factor

print(data)
data.close()


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    name: ASO Tuolumne basin machine learning and geostatistics data set
    contains: SWE depth maps from 16 Airborne Snow Observatory lidar scans (2016-2019)  and "static" terrain parameters from the SRTM DEM and the National Land Cover Database
    location: Tuolumne basin (around Hetch Hetch Reservoir), in the Californian Sierra Nevada
    time period: Multiple flights in the snow seasons 2016, 2017, 2018, and 2019
    georeferencing: Pixels/cells are regularly spaced in zone 11S of the UTM projection based on the WGS84 ellipsoid.
    convention: By convention, the first pixel (i.e.: row index i=1, column index j=1) is in the upper left corner of the image.
    dimensions: The image stack "D" (SWE maps) have 3 dimensions [dim1=rows=y_i=northing, dim2=columns=x_j=easting, dim3=t_n=time of flight]
    Ground sampling distance: GSD=100 meters (spatial resolution)
    easting coordinate: The easting p

In [1]:
if __name__ == "__main__":
    
    def display_image(image,title):
        plt.figure(figsize = [7,7])
        cm=plt.imshow(image)
        plt.title(title)
        plt.ylabel('Row index, i')
        plt.xlabel('Column index, j')
        plt.colorbar()
        plt.axis('equal')
        plt.savefig('figs/'+title + '_map.png')
        #plt.show()


    display_image(z,'Elevation')
    display_image(northness,'Northness')
    display_image(eastness,'Eastness')
    display_image(cc,'Canopy_cover')
    display_image(slp,'Slope')
    display_image(svf,'Sky_view_factor')


    #for i in range(D.shape[2]):
    #    display_image(D[:,:,i],i)
    plt.close('all')
  

NameError: name 'z' is not defined